In [3]:
import SimpleITK as sitk
import os

# 設定原始影像資料夾與保存裁切後影像的資料夾路徑
input_folder = '/home/jupyter-atticus453/Cardiac-CT-Image-10.7717/dataset/Epicardial_test_NC'
output_folder = '/home/jupyter-atticus453/Cardiac-CT-Image-10.7717/dataset/Epicardial_test_NC2'
os.makedirs(output_folder, exist_ok=True)

# 批次處理影像的範圍
start_idx = 201
end_idx = 220

for i in range(start_idx, end_idx + 1):
    # 構建文件名，例如 NC001_NC.nrrd, NC002_NC.nrrd 等
    filename = f"NC{i:03d}_NC.nrrd"
    #filename = f"NC-UofT04.nrrd"
    input_path = os.path.join(input_folder, filename)
    output_path = os.path.join(output_folder, filename)
    
    # 確認檔案存在
    if not os.path.exists(input_path):
        print(f"檔案 {filename} 不存在，跳過...")
        continue
    
    # 讀取影像
    image = sitk.ReadImage(input_path)
    image_array = sitk.GetArrayFromImage(image)
    
    # 裁切 HU 值範圍：低於 -1000 設為 -1000，高於 500 設為 500
    image_array = image_array.clip(min=-1000, max=500)
    
    # 將裁切後的數組轉換回 SimpleITK 影像格式
    clipped_image = sitk.GetImageFromArray(image_array)
    clipped_image.SetSpacing(image.GetSpacing())
    clipped_image.SetOrigin(image.GetOrigin())
    clipped_image.SetDirection(image.GetDirection())
    
    # 保存裁切後的影像
    sitk.WriteImage(clipped_image, output_path)
    print(f"已處理並保存影像 {filename}")

print("批次處理完成！")


檔案 NC200_NC.nrrd 不存在，跳過...
已處理並保存影像 NC201_NC.nrrd
已處理並保存影像 NC202_NC.nrrd
已處理並保存影像 NC203_NC.nrrd
已處理並保存影像 NC204_NC.nrrd
已處理並保存影像 NC205_NC.nrrd
已處理並保存影像 NC206_NC.nrrd
已處理並保存影像 NC207_NC.nrrd
已處理並保存影像 NC208_NC.nrrd
已處理並保存影像 NC209_NC.nrrd
已處理並保存影像 NC210_NC.nrrd
已處理並保存影像 NC211_NC.nrrd
已處理並保存影像 NC212_NC.nrrd
已處理並保存影像 NC213_NC.nrrd
已處理並保存影像 NC214_NC.nrrd
已處理並保存影像 NC215_NC.nrrd
已處理並保存影像 NC216_NC.nrrd
已處理並保存影像 NC217_NC.nrrd
已處理並保存影像 NC218_NC.nrrd
已處理並保存影像 NC219_NC.nrrd
已處理並保存影像 NC220_NC.nrrd
批次處理完成！


In [5]:
import os
import shutil

# 設定來源資料夾和目標資料夾路徑
input_folder = '/home/jupyter-atticus453/Cardiac-CT-Image-10.7717/dataset/Epicardial_test_NC3'
output_folder = '/home/jupyter-atticus453/Cardiac-CT-Image-10.7717/dataset/Epicardial_test_NC2'
os.makedirs(output_folder, exist_ok=True)

# 批次處理影像的範圍
start_idx = 201
end_idx = 220

for i in range(start_idx, end_idx + 1):
    # 構建文件名，例如 NC001_gt-label, NC002_gt-label 等
    filename = f"NC{i:03d}_gt-label.nrrd"
    input_path = os.path.join(input_folder, filename)
    output_path = os.path.join(output_folder, filename)
    
    # 確認檔案存在
    if not os.path.exists(input_path):
        print(f"檔案 {filename} 不存在，跳過...")
        continue
    
    # 複製文件
    shutil.copy(input_path, output_path)
    print(f"已複製影像 {filename}")

print("批次複製完成！")


已複製影像 NC201_gt-label.nrrd
已複製影像 NC202_gt-label.nrrd
已複製影像 NC203_gt-label.nrrd
已複製影像 NC204_gt-label.nrrd
已複製影像 NC205_gt-label.nrrd
已複製影像 NC206_gt-label.nrrd
已複製影像 NC207_gt-label.nrrd
已複製影像 NC208_gt-label.nrrd
已複製影像 NC209_gt-label.nrrd
已複製影像 NC210_gt-label.nrrd
已複製影像 NC211_gt-label.nrrd
已複製影像 NC212_gt-label.nrrd
已複製影像 NC213_gt-label.nrrd
已複製影像 NC214_gt-label.nrrd
已複製影像 NC215_gt-label.nrrd
已複製影像 NC216_gt-label.nrrd
已複製影像 NC217_gt-label.nrrd
已複製影像 NC218_gt-label.nrrd
已複製影像 NC219_gt-label.nrrd
已複製影像 NC220_gt-label.nrrd
批次複製完成！


In [3]:
import SimpleITK as sitk

def resample_nrrd(input_path, output_path, target_size=(256, 256), target_spacing=(0.976562, 0.976562, 2.5)):
    # 讀取 NRRD 影像
    image = sitk.ReadImage(input_path)
    
    # 獲取原始影像的尺寸和 spacing
    original_size = image.GetSize()  # (X, Y, Z)
    original_spacing = image.GetSpacing()  # (X, Y, Z)
    
    # 計算 X 和 Y 軸的縮放比例，使其等比例縮小到 256 x 256
    scale_factor_x = original_size[0] / target_size[0]
    scale_factor_y = original_size[1] / target_size[1]
    
    # 計算新的 spacing，使影像在 X 和 Y 軸等比例縮小到 256 x 256
    new_spacing = (original_spacing[0] * scale_factor_x,
                   original_spacing[1] * scale_factor_y,
                   target_spacing[2])  # Z 軸 spacing 使用目標 spacing
    
    # 計算新的尺寸，確保 Z 軸的 spacing 和尺寸按照目標 spacing 調整
    new_size = list(target_size) + [int(round(original_size[2] * (original_spacing[2] / target_spacing[2])))]
    
    # 設置 Resample 過程
    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(new_spacing)
    resample.SetSize(new_size)
    resample.SetOutputDirection(image.GetDirection())
    resample.SetOutputOrigin(image.GetOrigin())
    resample.SetInterpolator(sitk.sitkLinear)
    
    # 進行重新取樣
    resampled_image = resample.Execute(image)
    
    # 保存結果
    sitk.WriteImage(resampled_image, output_path)
    print(f"已成功將影像等比例縮小並重新取樣至 {output_path}")

# 使用範例
input_file = "/home/jupyter-atticus453/Cardiac-CT-Image-10.7717/333.nrrd"  # 修改成你的檔案路徑
output_file = "/home/jupyter-atticus453/Cardiac-CT-Image-10.7717/34.nrrd"  # 儲存的檔案路徑
resample_nrrd(input_file, output_file)


已成功將影像等比例縮小並重新取樣至 /home/jupyter-atticus453/Cardiac-CT-Image-10.7717/34.nrrd


In [4]:
import SimpleITK as sitk

# 讀取 NRRD 檔案
input_file = "/home/jupyter-atticus453/Cardiac-CT-Image-10.7717/333.nrrd"  # 修改成你的檔案路徑
output_file = "/home/jupyter-atticus453/Cardiac-CT-Image-10.7717/335.nrrd"  # 儲存的檔案路徑

# 使用 SimpleITK 讀取影像
image = sitk.ReadImage(input_file)

# 目標尺寸
new_size = [256, 256, 64]

# 計算新的像素間距
old_size = image.GetSize()
old_spacing = image.GetSpacing()
new_spacing = [
    old_spacing[0] * (old_size[0] / new_size[0]),
    old_spacing[1] * (old_size[1] / new_size[1]),
    old_spacing[2] * (old_size[2] / new_size[2]),
]

# 使用 SimpleITK 的重新取樣功能
resampler = sitk.ResampleImageFilter()
resampler.SetOutputSpacing(new_spacing)
resampler.SetSize(new_size)
resampler.SetOutputDirection(image.GetDirection())
resampler.SetOutputOrigin(image.GetOrigin())
resampler.SetInterpolator(sitk.sitkLinear)

# 重新取樣影像
resampled_image = resampler.Execute(image)

# 儲存重新取樣後的影像
sitk.WriteImage(resampled_image, output_file)

print(f"影像已重新取樣並儲存為: {output_file}")


影像已重新取樣並儲存為: /home/jupyter-atticus453/Cardiac-CT-Image-10.7717/335.nrrd
